load pointcloud -> crop -> convex hull -> save as URDF -> load into Drake

In [ ]:
from pathlib import Path
from typing import List

import numpy as np
import open3d as o3d
import rerun as rr
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud
from airo_models.primitives.mesh import mesh_urdf_path
from airo_models.primitives.box import box_urdf_path
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from cloth_tools.bounding_boxes import BBOX_CLOTH_IN_THE_AIR, bbox_to_mins_and_sizes
from cloth_tools.dataset.format import load_competition_input_sample
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.drake.visualization import add_meshcat_triad, publish_dual_arm_joint_path, publish_ik_solutions
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.point_clouds.operations import filter_and_crop_point_cloud
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from ur_analytic_ik import ur5e

data_dir = Path("../data")
dataset_dir = data_dir / "dataset_0000"

In [ ]:
sample = load_competition_input_sample(dataset_dir, sample_index=0)

confidence_map = sample.confidence_map
point_cloud_in_camera = sample.point_cloud

In [ ]:
pcd_in_camera = point_cloud_to_open3d(point_cloud_in_camera)  # X_C_PC, need X_W_C

X_W_C = sample.camera_pose_in_world  # X_LCB_C (camera pose in the left-arm base frame)
pcd = pcd_in_camera.transform(X_W_C)  # transform to world frame

In [ ]:
point_cloud = open3d_to_point_cloud(pcd)

confidence_threshold = 1.0
confidence_mask = (confidence_map <= confidence_threshold).reshape(-1)  # Threshold and flatten
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)

bbox = BBOX_CLOTH_IN_THE_AIR

point_cloud_cropped = filter_and_crop_point_cloud(point_cloud, confidence_map, bbox)

In [ ]:
pcd_cropped = point_cloud_to_open3d(point_cloud_cropped)

bbox_o3d = o3d.geometry.AxisAlignedBoundingBox(*bbox)
bbox_o3d.color = (1.0, 0.0, 1.0)

o3d.visualization.draw_geometries([pcd_cropped.to_legacy(), bbox_o3d])

In [ ]:
hull = pcd_cropped.compute_convex_hull()

In [ ]:
o3d.visualization.draw_geometries([pcd_cropped.to_legacy(), hull.to_legacy(), bbox_o3d])

In [ ]:
# make the hull a bit bigger to avoid collisions
hull.scale(1.2, hull.get_center())

In [ ]:
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull.to_legacy())
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcd_cropped.to_legacy(), hull_ls, bbox_o3d])

In [ ]:
window_name = "convex_hull"
rr.init(window_name, spawn=True)

In [ ]:
rr_point_cloud = rr.Points3D(positions=point_cloud_filtered.points, colors=point_cloud_filtered.colors)
rr.log("world/point_cloud", rr_point_cloud)

In [ ]:
bbox = BBOX_CLOTH_IN_THE_AIR
bbox_color = (122, 173, 255)  # blue
bbox_mins, bbox_sizes = bbox_to_mins_and_sizes(bbox)
rr_bbox = rr.Boxes3D(mins=bbox_mins, sizes=bbox_sizes, colors=bbox_color)
rr.log("world/bbox", rr_bbox)

In [ ]:
hull.compute_vertex_normals()

In [ ]:
hull_color = (1.0, 0.2, 0.0, 0.5)

rr_mesh_material = rr.Material(hull_color)
rr_mesh = rr.Mesh3D(
    vertex_positions=hull.vertex.positions.numpy(),
    vertex_normals=hull.vertex.normals.numpy(),
    indices=hull.triangle.indices.numpy(),
    mesh_material=rr_mesh_material,
)

rr.log("world/hull", rr_mesh)

In [ ]:
# save hull as .obj
hull_path = data_dir / "hull.obj"
o3d.t.io.write_triangle_mesh(str(hull_path), hull)

In [ ]:
import tempfile

file = tempfile.NamedTemporaryFile(prefix="cloth_hull_", suffix=".obj", delete=False)
filename = file.name
filename

In [ ]:
o3d.t.io.write_triangle_mesh(filename, hull)

In [ ]:
hull_loaded = o3d.t.io.read_triangle_mesh(filename)
# hull_loaded.vertex.normals

In [ ]:
hull_legacy = hull_loaded.to_legacy()
hull_legacy.has_vertex_normals()
hull_legacy.compute_vertex_normals()
# hull_legacy.vertex_normals = o3d.utility.Vector3dVector([])

In [ ]:
o3d.visualization.draw_geometries([hull_legacy])

In [ ]:
path = mesh_urdf_path(filename, "cloth_hull")
path

In [ ]:
safety_wall = box_urdf_path((0.2, 0.05, 0.3), "safety_wall")

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)

# Add the cloth hull to the scene
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
hull_index = parser.AddModels(path)[0]
safety_wall_index = parser.AddModels(safety_wall)[0]

world_frame = plant.world_frame()
hull_frame = plant.GetFrameByName("base_link", hull_index)
safety_wall_frame = plant.GetFrameByName("base_link", safety_wall_index)

safety_wall_transform = RigidTransform(p=[0.0, -0.15, 0.85])  # move the safety wall to the right of the table
plant.WeldFrames(world_frame, hull_frame)
plant.WeldFrames(world_frame, safety_wall_frame, safety_wall_transform)

diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
start_joints_left = np.deg2rad([0, -100, -20, -90, 90, 0])
start_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])
# goal_joints_right = np.deg2rad([-100, -116, -110, -133, 40, 0])

plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, start_joints_left)
plant.SetPositions(plant_context, arm_right_index, start_joints_right)
diagram.ForcedPublish(context)

In [ ]:
Z = np.array([1, 0, 0])
X = np.array([0, 1, 0])
Y = np.cross(Z, X)
orientation = np.column_stack([X, Y, Z])

position = np.array([-0.2, -0.15, 0.24])

pregrasp_pose = np.identity(4)
pregrasp_pose[:3, :3] = orientation
pregrasp_pose[:3, 3] = position

add_meshcat_triad(meshcat, "pregrasp_pose", X_W_Triad=pregrasp_pose, length=0.1)

In [ ]:
from functools import partial


tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175


def inverse_kinematics_in_world_fn(
    tcp_pose: HomogeneousMatrixType, X_W_CB: HomogeneousMatrixType
) -> List[JointConfigurationType]:
    X_W_TCP = tcp_pose
    X_CB_W = np.linalg.inv(X_W_CB)
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_CB_W @ X_W_TCP, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions


X_W_LCB = sample.left_arm_pose_in_world
X_W_RCB = sample.right_arm_pose_in_world

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB)


solutions_right = inverse_kinematics_right_fn(pregrasp_pose)
publish_ik_solutions(solutions_right, 2.0, meshcat, diagram, context, arm_right_index)

In [ ]:
planner = DualArmOmplPlanner(
    collision_checker.CheckConfigCollisionFree, inverse_kinematics_left_fn, inverse_kinematics_right_fn
)
path = planner.plan_to_tcp_pose(start_joints_left, start_joints_right, None, pregrasp_pose)

In [ ]:
publish_dual_arm_joint_path(path, 5.0, meshcat, diagram, context, arm_left_index, arm_right_index)